In [1]:
#the basics
import numpy as np
import pandas as pd
from itertools import product

#viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette='colorblind')

#modeling tools
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE

#My modules
import wrangle
import utils

# Model Prep

###### Acquire and prepare the data

In [2]:
tr, te, val = wrangle.wrangle_zillow(include_zip=True,val_ratio=.15,test_ratio=.15)

In [3]:
tr.shape

(35072, 341)

In [4]:
tr.columns[0:12]

Index(['value', 'zipcode', 'county', 'bed', 'bath', 'sf', 'sf_per_bed',
       'yearbuilt', 'Orange_CA', 'Ventura_CA', '95983', '95984'],
      dtype='object')

##### Create scaled data subsets

In [5]:
tr_mm, te_mm, val_mm = wrangle.scale_zillow(tr,te,val)
tr_st, te_st, val_st = wrangle.scale_zillow(tr,te,val,kind='standard')

In [6]:
tr_mm.columns[:12]

Index(['value', 'zipcode', 'county', 'bed', 'bath', 'sf', 'sf_per_bed',
       'yearbuilt', 'Orange_CA', 'Ventura_CA', '95983', '95984'],
      dtype='object')

##### Create model input subsets

In [7]:
#split the train subset
#all contains zip, X_tr contains all but zip, y_tr is target
X_tr_mm_all = tr_mm.iloc[:,3:]
X_tr_st_all = tr_st.iloc[:,3:]

X_tr_mm = tr_mm.iloc[:,3:10]
X_tr_st = tr_st.iloc[:,3:10]

#test
X_te_mm_all = te_mm.iloc[:,3:]
X_te_st_all = te_st.iloc[:,3:]

X_te_mm = te_mm.iloc[:,3:10]
X_te_st = te_st.iloc[:,3:10]

#val
X_val_mm_all = val_mm.iloc[:,3:]
X_val_st_all = val_st.iloc[:,3:]

X_val_mm = val_mm.iloc[:,3:10]
X_val_st = val_st.iloc[:,3:10]

#target variables aren't scaled, so always the same
y_tr = tr.value
y_te = te.value
y_val = val.value

In [8]:
X_tr_mm.head(2)

,bed,bath,sf,sf_per_bed,yearbuilt,Orange_CA,Ventura_CA
17424,0.5,0.285714,0.221761,0.246835,0.541985,0,0
984,0.5,0.142857,0.144897,0.136634,0.595420,1,0


##### Feature Selection

In [9]:
#let's look at 
for k in [2,3,4,5,15]:
    #Create selector
    f_selector_mm = f_selector_st = SelectKBest(f_regression, k=k)
    #fit to train
    f_selector_mm.fit(X_tr_mm_all, y_tr)
    #Let's look at the chosen columns
    print(X_tr_mm_all.columns[f_selector_mm.get_support()])

Index(['bath', 'sf'], dtype='object')
Index(['bath', 'sf', 'sf_per_bed'], dtype='object')
Index(['bed', 'bath', 'sf', 'sf_per_bed'], dtype='object')
Index(['bed', 'bath', 'sf', 'sf_per_bed', 'yearbuilt'], dtype='object')
Index(['bed', 'bath', 'sf', 'sf_per_bed', 'yearbuilt', 'Orange_CA', '96117',
       '96449', '96940', '96966', '96975', '96978', '96989', '97318', '97328'],
      dtype='object')


**NOTES:** Based off this, I want to run linear regression on:
- ['bath','sf']
- ['bed','bath','sf']
- top 15, but dropping sf_per_bed:
  - ['bed', 'bath', 'sf', 'yearbuilt', 'Orange_CA', '96030',
       '96050', '96086', '96116', '96117', '96120',...]
       

##### Get a baseline

In [10]:
#create df with actual values
tr_res = pd.DataFrame(tr.value)
tr_res.rename(columns={'value':'actual'},inplace=True)
val_res = pd.DataFrame(val.value)
val_res.rename(columns={'value':'actual'},inplace=True)
te_res = pd.DataFrame(te.value)
te_res.rename(columns={'value':'actual'},inplace=True)
#Create potential baselines
tr_res['base_mean'] = val_res['base_mean'] = tr.value.mean()
tr_res['base_median'] = val_res['base_median'] = tr.value.median()


tr_res.describe().T

,count,mean,std,min,25%,50%,75%,max
actual,35072.0,443581.03547,3.520284e+05,10504.00000,189010.75000,363924.50000,587346.75000,1.996000e+06
base_mean,35072.0,443581.03547,3.170034e-07,443581.03547,443581.03547,443581.03547,443581.03547,4.435810e+05
base_median,35072.0,363924.50000,0.000000e+00,363924.50000,363924.50000,363924.50000,363924.50000,3.639245e+05


In [11]:
for c in tr_res.columns[1:]:
    score = utils.rmse(tr_res.actual,tr_res[c])
    score2 = utils.rmse(val_res.actual,val_res[c])
    print(f'{c} has a rmse of {score:.0f} on train and {score2:.0f} on validate')

base_mean has a rmse of 352023 on train and 356032 on validate
base_median has a rmse of 360923 on train and 365318 on validate


mean performs slightly better on both train and validate >> use it.

In [12]:
#set it as official baseline
tr_res['baseline'] = val_res['baseline'] = tr.value.mean()
#drop the other columns we don't need anymore
tr_res.drop(columns=['base_mean','base_median'],inplace=True)
val_res.drop(columns=['base_mean','base_median'],inplace=True)

#take a peek
tr_res.describe().T

,count,mean,std,min,25%,50%,75%,max
actual,35072.0,443581.03547,3.520284e+05,10504.00000,189010.75000,363924.50000,587346.75000,1.996000e+06
baseline,35072.0,443581.03547,3.170034e-07,443581.03547,443581.03547,443581.03547,443581.03547,4.435810e+05


##### Create function to grab top k_features

In [13]:
def select_kbest(X,y,k):
    '''
    Uses sklearn.feature_selection.SelectKBest to select top k features.
    
    Returns: List corresp
    Inputs: 
      (R) X: Pandas Dataframe of features and values
      (R) y: target variable
      (R) k: number of features to select
    '''
    #Create feature selector & fit
    f_selector = SelectKBest(f_regression,k=k).fit(X,y)
    # Boolean mask of which columns are selected
    f_mask = f_selector.get_support()
    #get list of top features
    k_features = X.columns[f_mask].tolist()
    #return features as list
    return k_features

# Modeling

- OLS - LinearRegression:
  - normalize - true
  - scaler: standard, min/max
  - features: all, all but zip, top 3
- Lasso + LARS 
  - NOTE: 
    - Does feature selection
    - Y should be normal
    - alpha penalizes more features
  - alpha: .25, .5, .75, 1, 1.5
- Generalized Linear MOdel - TweedieRegressor
  - power: 1 (poisson) 
  - power: 2 (gamma), link: 'log'
- Polynomialfeature >> linearregression
  - NOTE: not expect any benefit from this
  - degree: 2


In [14]:
#initialize model dictionary and performance
model_dict ={}
mod_perf = pd.DataFrame()

#####  Linear Regression

In [15]:
#perform Linear regression on 
for k in [2,3,4,5,6,15,20,'all']:
    #generate model name
    model_name = 'lr_mm_k'+str(k)
    #get kbest features
    k_best = select_kbest(X_tr_mm_all,y_tr,k)
    #create and fit model on train
    model = LinearRegression(normalize=True).fit(X_tr_mm_all[k_best],y_tr)
    #add model to dictionary
    model_dict[model_name] = {
        "model_name": model_name,
        "model": model,
        "kbest": k_best}
    #generate and store model predictions
    tr_res[model_name] = model.predict(X_tr_mm_all[k_best])

#take a peek
tr_res.head()

,actual,baseline,lr_mm_k2,lr_mm_k3,lr_mm_k4,lr_mm_k5,lr_mm_k6,lr_mm_k15,lr_mm_k20,lr_mm_kall
17424,379950,443581.03547,488255.659648,468506.682719,456755.447907,470483.491575,457089.731735,456439.941723,457335.198520,494079.406790
984,69381,443581.03547,328862.589871,284252.771385,255233.044404,253272.018888,296987.456130,288555.727740,286492.397392,212888.055475
34863,159623,443581.03547,490284.938964,456230.833690,439777.532458,459814.077120,450640.940611,444829.602235,444254.129730,351311.458371
17025,379262,443581.03547,291312.471232,277415.056712,289648.177529,292296.136022,280173.739764,287213.721918,293096.779952,251172.941791
10245,361154,443581.03547,370297.203542,328396.872939,302602.752567,299390.900866,281259.054444,291046.938783,303868.706859,224238.481277


In [16]:
#Now let's see how they did
for model_name in tr_res.columns[1:]:
    mod_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res[model_name]))

mod_perf

,baseline,lr_mm_k2,lr_mm_k3,lr_mm_k4,lr_mm_k5,lr_mm_k6,lr_mm_k15,lr_mm_k20,lr_mm_kall
rmse,352023.0,287097.0,284043.0,283053.0,282629.0,281435.0,275116.0,271511.0,245130.0


In [17]:
#repeat with other scale
for k in [2,3,4,5,6,15,20]:
    #generate model name
    model_name = 'lr_st_k'+str(k)
    #get kbest features
    k_best = select_kbest(X_tr_st_all,y_tr,k)
    #create and fit model on train
    model = LinearRegression(normalize=True).fit(X_tr_st_all[k_best],y_tr)
    #add model to dictionary
    model_dict[model_name] = {
        "model_name": model_name,
        "model": model,
        "kbest": k_best}
    #generate and store model predictions
    tr_res[model_name] = model.predict(X_tr_st_all[k_best])
    #gather model performance
    mod_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res[model_name]))

#take a peek at performance
mod_perf

,baseline,lr_mm_k2,lr_mm_k3,lr_mm_k4,lr_mm_k5,lr_mm_k6,lr_mm_k15,lr_mm_k20,lr_mm_kall,lr_st_k2,lr_st_k3,lr_st_k4,lr_st_k5,lr_st_k6,lr_st_k15,lr_st_k20
rmse,352023.0,287097.0,284043.0,283053.0,282629.0,281435.0,275116.0,271511.0,245130.0,287097.0,284043.0,283053.0,282629.0,281435.0,275116.0,271511.0


In [18]:
mod_perf.columns

Index(['baseline', 'lr_mm_k2', 'lr_mm_k3', 'lr_mm_k4', 'lr_mm_k5', 'lr_mm_k6',
       'lr_mm_k15', 'lr_mm_k20', 'lr_mm_kall', 'lr_st_k2', 'lr_st_k3',
       'lr_st_k4', 'lr_st_k5', 'lr_st_k6', 'lr_st_k15', 'lr_st_k20'],
      dtype='object')

In [19]:
#both scaling methods had the same results - just use min max moving forward
#drop the standard scaling results form tr_res and mod_perf
drp_cols = ['lr_st_k2', 'lr_st_k3', 'lr_st_k4','lr_st_k5', 'lr_st_k6', 'lr_st_k15', 'lr_st_k20']
mod_perf.drop(columns=drp_cols,inplace=True)
tr_res.drop(columns=drp_cols,inplace=True)

##### Generalized Linear MOdel - TweedieRegressor
- power: 1 (poisson)
- power: 2 (gamma), link: 'log'
- alphas: 0,.25,.5,.75,1,2

In [20]:
glm_mod_perf = pd.DataFrame()
tr_res_glm_mm = pd.DataFrame()
ks = [2,3,4,5,6,15]
alphas = [0,.5,1]
params = product(ks,alphas)
#perform Generalized Linear Modeul using Poisson
for pair in params:
    k = pair[0]
    a = pair[1]
    #generate model name
    model_name = 'pow1_k'+str(k)+'_a'+str(a)
    #get kbest features
    k_best = select_kbest(X_tr_mm_all,y_tr,k)
    #create and fit model on train
    model = TweedieRegressor(power=1,alpha=a,max_iter=300).fit(X_tr_mm_all[k_best],y_tr)
    #add model to dictionary
    model_dict[model_name] = {
        "model_name": model_name,
        "model": model,
        "kbest": k_best}
    #generate and store model predictions
    tr_res_glm_mm[model_name] = model.predict(X_tr_mm_all[k_best])
    #gather model performance
    glm_mod_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res_glm_mm[model_name]))
    glm_mod_perf.loc['r2',model_name] = round(r2_score(tr_res.actual,tr_res_glm_mm[model_name]),3)


In [21]:
#perform Generalized Linear Modeul using Gamma - log
params = product(ks,alphas) #Need to recreate this for some reason - time permitting: determine why
for pair in params:
    k = pair[0]
    a = pair[1]
    #generate model name
    model_name = 'pow2_linkLog'+str(k)+'_a'+str(a)
    #get kbest features
    k_best = select_kbest(X_tr_mm_all,y_tr,k)
    #create and fit model on train
    model = TweedieRegressor(power=2,alpha=a,link='log',max_iter=300).fit(X_tr_mm_all[k_best],y_tr)
    #add model to dictionary
    model_dict[model_name] = {
        "model_name": model_name,
        "model": model,
        "kbest": k_best}
    #generate and store model predictions
    tr_res_glm_mm[model_name] = model.predict(X_tr_mm_all[k_best])
    #gather model performance
    glm_mod_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res_glm_mm[model_name]))
    glm_mod_perf.loc['r2',model_name] = round(r2_score(tr_res.actual,tr_res_glm_mm[model_name]),3)

In [22]:
glm_mod_perf.T.sort_values(by='rmse',ascending=True)

,rmse,r2
pow1_k15_a1,279740.0,0.369
pow1_k15_a0.5,279740.0,0.369
pow1_k15_a0,279740.0,0.369
pow1_k6_a0.5,286513.0,0.338
pow1_k6_a0,286513.0,0.338
pow1_k6_a1,286514.0,0.338
pow1_k4_a0,287510.0,0.333
pow1_k4_a0.5,287511.0,0.333
pow1_k4_a1,287512.0,0.333
pow2_linkLog15_a0,287776.0,0.332


##### LassoLars


In [23]:
alphas = [.1,.25, .5, .75, 1, 1.5]
ll_mod_perf = pd.DataFrame()
tr_res_ll_mm = pd.DataFrame()
for a in alphas:
    #generate model name
    model_name = 'll_a'+str(a)
    #create and fit model on train
    model = LassoLars(alpha=a).fit(X_tr_mm_all,y_tr)
    #add model to dictionary
    model_dict[model_name] = {
        "model_name": model_name,
        "model": model,
        "kbest": X_tr_mm_all.columns}
    #generate and store model predictions
    tr_res_ll_mm[model_name] = model.predict(X_tr_mm_all)
    #gather model performance
    ll_mod_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res_ll_mm[model_name]))
    ll_mod_perf.loc['r2',model_name] = round(r2_score(tr_res.actual,tr_res_ll_mm[model_name]),3)


In [24]:
ll_mod_perf.T.sort_values(by='rmse',ascending=True)

,rmse,r2
ll_a0.1,245153.0,0.515
ll_a0.25,245185.0,0.515
ll_a0.5,245207.0,0.515
ll_a0.75,245226.0,0.515
ll_a1,245250.0,0.515
ll_a1.5,245312.0,0.514


**NOTES:** Looking at the train performance, I want to run the following models against validate:
- 'll_a0.1','ll_a1','pow1_k15_a1','pow1_k6_a1','pow2_linkLog15_a0','lr_st_kall','lr_mm_k6'

In [25]:
#models to run over validate
val_models = ['ll_a0.1','ll_a1','pow1_k15_a1','pow1_k6_a1','pow2_linkLog15_a0','lr_mm_kall','lr_mm_k6']
combined_perf = pd.DataFrame()
tr_res2 = pd.DataFrame()
#create new model performance dataframe
mod_perf_comp = pd.DataFrame()
for model_name in val_models:
    #grab model & kbest from dictionary:
    model = model_dict[model_name]['model']
    k_best = model_dict[model_name]['kbest']
    #generate prediction on tr & val dataset - adds new column
    tr_res2[model_name] = model.predict(X_tr_mm_all[k_best])
    val_res[model_name] = model.predict(X_val_mm_all[k_best])
    #gather model performance on tr & val
    combined_perf.loc['rmse',model_name] = round(utils.rmse(tr_res.actual,tr_res2[model_name]))
    combined_perf.loc['r2',model_name] = round(r2_score(tr_res.actual,tr_res2[model_name]),3)
    combined_perf.loc['rmse_val',model_name] = round(utils.rmse(val_res.actual,val_res[model_name]))
    combined_perf.loc['r2_val',model_name] = round(r2_score(val_res.actual,val_res[model_name]),3)
    #determine improvement
    combined_perf.loc['rmse_diff',model_name] = combined_perf.loc['rmse_val',model_name] - combined_perf.loc['rmse',model_name]
    combined_perf.loc['r2_diff',model_name] = combined_perf.loc['r2_val',model_name] - combined_perf.loc['r2',model_name]


In [26]:
combined_perf.T.sort_values(by='rmse')

,rmse,r2,rmse_val,r2_val,rmse_diff,r2_diff
lr_mm_kall,245130.0,0.515,249194.0,0.510,4064.0,-0.005
ll_a0.1,245153.0,0.515,249144.0,0.510,3991.0,-0.005
ll_a1,245250.0,0.515,249213.0,0.510,3963.0,-0.005
pow1_k15_a1,279740.0,0.369,281154.0,0.376,1414.0,0.007
lr_mm_k6,281435.0,0.361,283096.0,0.368,1661.0,0.007
pow1_k6_a1,286514.0,0.338,287833.0,0.346,1319.0,0.008
pow2_linkLog15_a0,287776.0,0.332,287890.0,0.346,114.0,0.014


**NOTES:** All of the models did well going from train to validate.  The ones with the best performance RMSE and R2 had the greatest loss when moving to validate, however it was still only less than 2% loss in performance.  The top 3 performed very similarly.  Moving forward, I'll use the lasso lars with an alpha of 1 on the test dataset

In [27]:
model = model_dict['ll_a1']['model']
k_best = model_dict['ll_a1']['kbest']

#now see how it performs on test
te_pred = model.predict(X_te_mm_all[k_best])

print(f'Test RMSE: {utils.rmse(y_te,te_pred)}')
print(f'Test r2: {r2_score(y_te,te_pred)}')


Test RMSE: 247637.51980104964
Test r2: 0.5076425676195204


**NOTES:** The Lasso Lars model performed well against the test dataset as well, with an r2 of .51 and RMSE of 247,637